# timmを使用した転移学習モデル

### 参考URL→ https://zenn.dev/piment/articles/4ff3b6dfd73103

In [295]:
import timm
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn as nn
import os
import matplotlib.pyplot as plt

In [296]:
# timm.list_models(pretrained=True)

In [297]:
#in_chans=1でグレイスケール
model = timm.create_model('resnet18d',pretrained=True,num_classes = 2, in_chans = 1)
# model = timm.create_model('resnet18d', pretrained=True, num_classes = 2)

In [298]:
# freeze layers except last layer
for param in model.parameters():
    param.requires_grad = False

last_layer = list(model.children())[-1]
print(f'except last layer: {last_layer}')
for param in last_layer.parameters():
    param.requires_grad = True

except last layer: Linear(in_features=512, out_features=2, bias=True)


In [299]:
# print(model)

In [300]:
#追加する層
net_seq=nn.Sequential(
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=True),
    nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=True),
    nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True)
)

In [301]:
net_seq

Sequential(
  (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
)

In [302]:
#層を追加
# model.add_module('layer5', nn.Identity())
# model.layer5 = nn.Sequential(net_seq, model.layer5)
model.layer4.add_module('add_layer', net_seq)
# model.layer4 = nn.Sequential(net_seq, model.layer4)

In [303]:
model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
   

# 学習させる画像 https://cgpipeliner.info/2022/04/29/cnn-datalorder/#st-toc-h-3

In [304]:
for i in os.listdir('cut_roundeye/'):
    print(i)

nmirror
mirror


# trainデータを作成した後、7割をtrain, 3割をvalに分けてDataLoaderとする

In [305]:
# import torch
# import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
# import pytorch_lightning as pl

#シード値の固定
# pl.seed_everything(0)

# 学習データのパス
data_path = 'new_TrainDataset'

# バッチサイズ
batch_size = 256

# オーグメンテーション
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
#     """Convert a color image to grayscale and normalize the color range to [0,1]."""
])

# データセットの作成
dataset = datasets.ImageFolder(data_path, transform)


# 学習データに使用する割合
n_train_ratio = 70

# 割合から個数を出す
n_train = int(len(dataset) * n_train_ratio / 100)
n_val   = int(len(dataset) - n_train)


# 学習データと検証データに分割
train, val = torch.utils.data.random_split(dataset, [n_train, n_val])

# 
# Data Loader
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)

In [306]:
print('データローダのタイプ', type(train_loader))
print('全体のデータ量 : ',len(dataset))
print('イテレーション : ', len(train_loader))

データローダのタイプ <class 'torch.utils.data.dataloader.DataLoader'>
全体のデータ量 :  22769
イテレーション :  62


### 画像とラベルを取り出す

In [307]:
# データロダからバッチを取り出す
image_list, label_list = next(iter(train_loader))

print('画像の枚数   : ', len(image_list))
print('ラベルの個数 : ', len(label_list))
print('画像        : ', image_list[0].shape)
print('ラベル      : ', label_list[0])

画像の枚数   :  256
ラベルの個数 :  256
画像        :  torch.Size([1, 64, 64])
ラベル      :  tensor(0)


# 学習　参考→ https://htomblog.com/pthon-timm

In [ ]:
from timm.utils import AverageMeter
from tqdm import tqdm

# 最適化手法
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

# 損失関数
criterion = torch.nn.CrossEntropyLoss()

# ログ記録用の変数
history = {"train": [], "test": []}

# 学習回数
for epoch in range(50):
    print("\nEpoch:", epoch)

    # 学習
    model.train()
    train_loss = AverageMeter()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        image = batch[0] #(batch_size, channel, size, size)
        label = batch[1] #(batch_size)
#         preds = model(image) #(batch_size, num_class)
        preds = model(image)
        loss = criterion(preds, label)
        loss.backward()
        optimizer.step()
        train_loss.update(val = loss.item(), n = len(image))

    # 検証
    model.eval()
    test_loss = AverageMeter()
    with torch.no_grad():
        for batch in tqdm(val_loader):
            image = batch[0] #(batch_size, channel, size, size)
            label = batch[1] #(batch_size)
            preds = model(image) #(batch_size, num_class)
            loss = criterion(preds, label)
            test_loss.update(val = loss.item(), n = len(image))

    # 誤差出力
    print(train_loss.avg)
    print(test_loss.avg)
    history["train"].append(train_loss.avg)
    history["test"].append(test_loss.avg)

  0%|          | 0/62 [00:00<?, ?it/s]


Epoch: 0


  0%|          | 0/62 [00:00<?, ?it/s]

0.21596265043462476
0.24824279940698982

Epoch: 1


  0%|          | 0/62 [00:00<?, ?it/s]

...

  0%|          | 0/62 [00:00<?, ?it/s]

0.0125836075148395
0.02782339860571055

Epoch: 6


  0%|          | 0/62 [00:00<?, ?it/s]

0.008833359757949027
0.02622386720263274

Epoch: 7


 52%|█████▏    | 14/27 [00:05<00:04,  2.68it/s]

In [ ]:
plt.plot(history["train"], label = "train")
plt.plot(history["test"], label = "test")
plt.legend()
plt.show()

# test画像

In [ ]:

# 学習データのパス
test_path = 'new_TestDataset'

# バッチサイズ
batch_size = 256

# オーグメンテーション
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# データセットの作成
test_dataset = datasets.ImageFolder(test_path, transform)

# Data Loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size)

In [ ]:
# plt.figure()
# plt.subplot(1, 2, 1)
# plt.imshow(data_path[0][0].permute(1, 2, 0))
# plt.title(train_loader[1][1])
# plt.subplot(1, 2, 2)
# plt.imshow(test_loader[0][0].permute(1, 2, 0))
# plt.title(test_loader[1][1])
# plt.show()

In [ ]:
import mediapipe

model.eval()
preds = []
labels = []

mp_drawing = mediapipe.solutions.drawing_utils
mp_drawing_styles = mediapipe.solutions.drawing_styles
mp_face_mesh = mediapipe.solutions.face_mesh

with torch.no_grad():
    for batch in tqdm(test_loader):
        image = batch[0]
        label = batch[1]
        
#         #Augmentationした画像データを読み込む
#         tmp = iter(batch)
#         images,labels = next(tmp)

#         #画像をtensorからnumpyに変換
#         images = images.numpy()

#         with mp_face_mesh.FaceMesh(max_num_faces=1,refine_landmarks=True,min_detection_confidence=0.7,min_tracking_confidence=0.7) as face_mesh:
#             results = face_mesh.process(images)
#             if results.multi_face_landmarks:
#                 for face_landmarks in results.multi_face_landmarks:
#                     #Face_Mesh

#                     for number, lm in enumerate(face_landmarks.landmark):
#                         #image_coo: 付け足す座標
#                         img_coo = [0.0] * 64
#                         ih, iw, ic = image.shape
#                         x, y  = int(lm.x*iw), int(lm.y*ih)
#                         z = lm.z
#                         #0, 4, 5, 6, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477の座標取得、6の時は目の周りの切り取りも行う

#                         if number == 0:
#                             counter = 0
#                             img_coo[0] = z
#                         elif number == 4:
#                             img_coo[1] = z
#                         elif number == 5:
#                             img_coo[2] = z
#                         elif number == 6:
#                             x1 = x - 120
#                             x2 = x + 120
#                             y1 = y - 50
#                             y2 = y + 20

#                             img_coo[3] = z

#                             image_cut = image[y1 : y2, x1: x2]

# #                         elif number >= 468 and number < 477:
# #                             img_coo[number - 464] = z
# #                         elif number == 477:
# #                             img_coo[14] = z
# #                             counter += 1

#                             if image.size == 0:
#                                 continue
#                             else:
#                                 image= cv2.cvtColor(image_cut, cv2.COLOR_BGR2GRAY)




        # 最も値の大きい列番号
        preds += model(image).numpy().argmax(axis = 1).tolist()
#         print(preds)
        # 答え
        labels += label.numpy().tolist()

In [ ]:
print(preds[:5])
print(labels[:5])

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, preds))

In [ ]:
# torch.save(model.state_dict(), 'model_resnet50_pre50.pth')
# torch.save(model, 'model_resnet50_pre50.pth')
torch.save(model.state_dict(), 'model_resnet18d_NoFrop_3_new_TrainDataset.pth')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm_pre = confusion_matrix(labels, preds)

In [ ]:
#fmt='d'で数字の指数表記をなくせる
sns.heatmap(cm_pre, annot=True, cmap='Blues', fmt='d')
plt.ylabel('Tdata')
plt.xlabel('predictions')
# plt.title(path)

# 瀬川さん

In [ ]:
# 学習データのパス
test_path = 'class_test_image_cut/segawa/'

# バッチサイズ
batch_size = 256

# オーグメンテーション
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# データセットの作成
test_dataset = datasets.ImageFolder(test_path, transform)

# Data Loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size)

In [ ]:
import mediapipe

model.eval()
preds = []
labels = []

mp_drawing = mediapipe.solutions.drawing_utils
mp_drawing_styles = mediapipe.solutions.drawing_styles
mp_face_mesh = mediapipe.solutions.face_mesh

with torch.no_grad():
    for batch in tqdm(test_loader):
        image = batch[0]
        label = batch[1]
        

        # 最も値の大きい列番号
        preds += model(image).numpy().argmax(axis = 1).tolist()
#         print(preds)
        # 答え
        labels += label.numpy().tolist()

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, preds))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm_pre = confusion_matrix(labels, preds)

In [ ]:
#fmt='d'で数字の指数表記をなくせる
sns.heatmap(cm_pre, annot=True, cmap='Blues', fmt='d')
plt.ylabel('Tdata')
plt.xlabel('predictions')
plt.title(test_path)

# Mom

In [ ]:
# 学習データのパス
test_path = 'class_test_image_cut/mom/'

# バッチサイズ
batch_size = 256

# オーグメンテーション
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# データセットの作成
test_dataset = datasets.ImageFolder(test_path, transform)

# Data Loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size)

In [ ]:
import mediapipe

model.eval()
preds = []
labels = []

mp_drawing = mediapipe.solutions.drawing_utils
mp_drawing_styles = mediapipe.solutions.drawing_styles
mp_face_mesh = mediapipe.solutions.face_mesh

with torch.no_grad():
    for batch in tqdm(test_loader):
        image = batch[0]
        label = batch[1]
        

        # 最も値の大きい列番号
        preds += model(image).numpy().argmax(axis = 1).tolist()
#         print(preds)
        # 答え
        labels += label.numpy().tolist()

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, preds))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm_pre = confusion_matrix(labels, preds)

In [ ]:
sns.heatmap(cm_pre, annot=True, cmap='Blues', fmt='d')
plt.ylabel('Tdata')
plt.xlabel('predictions')
plt.title(test_path)